In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import importlib

if importlib.util.find_spec('datasets') is None:
    !pip install -q torch==2.2.1 transformers==4.39.1 diffusers==0.27.2 torchvision==0.17.1 datasets==2.18.0


if importlib.util.find_spec("wandb") is None:
    !pip install -q wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.

In [3]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    # For wandb setting
    project = "IOAI_Task3"
    name = "arisa_030_016_alpha_1.5_yolo_loss"

    # Model setting
    base_model_name="lambdalabs/miniSD-diffusers"

    learning_rate = 2e-05
    resolution = 256
    max_train_steps = 4000
    train_batch_size = 8

    # for Yolo Loss
    # same as cv_scoring notebook
    yolo_loss_lambda = 1.5
    threshold = 0.6

    seed=42

    # for evaluation (generate the image in given prompts)
    eval_steps = 500
    prompts = ['A curious zebra standing tall in a lush African savanna at sunrise, with acacia tree,s in the background.',
        'Next to a medieval castle, a regal zebra observes the knights and a drawbridge.',
        'Wearing a scarf, a fashionable giraffe strolls through a bustling city street with skyscrapers.',
        'Running along a sandy beach, a playful giraffe enjoys the palm trees, ocean waves, and a bright sunset.',
        # 'By a serene lakeside, a relaxed bear drinks water with mountains and a clear blue sky in the background.',
        # 'In a snowy forest, a cozy bear stands under snow-covered trees, enjoying the gentle snowfall.',
        'Partially hidden in a dense tropical rainforest, an adventurous sheep peeks through leafy plants.',
        'A sleek sheep with modern accessories navigates a futuristic city with flying cars and neon lights.',
        'By a serene lakeside, a relaxed bird drinks water with mountains and a clear blue sky in the background.',
        'In a snowy forest, a cozy bird stands under snow-covered trees, enjoying the gentle snowfall.',
        'Partially hidden in a dense tropical rainforest, an adventurous bird peeks through leafy plants.',
        'A sleek bird with modern accessories navigates a futuristic city with flying cars and neon lights.',
        'By a serene lakeside, a relaxed cat drinks water with mountains and a clear blue sky in the background.',
        'In a snowy forest, a cozy cat stands under snow-covered trees, enjoying the gentle snowfall.',
        'Partially hidden in a dense tropical rainforest, an adventurous cat peeks through leafy plants.',
        'A sleek cat with modern accessories navigates a futuristic city with flying cars and neon lights.',
        'By a serene lakeside, a relaxed dog drinks water with mountains and a clear blue sky in the background.',
        'In a snowy forest, a cozy dog stands under snow-covered trees, enjoying the gentle snowfall.',
        'Partially hidden in a dense tropical rainforest, an adventurous dog peeks through leafy plants.',
        'A sleek dog with modern accessories navigates a futuristic city with flying cars and neon lights.',
        'By a serene lakeside, a relaxed horse drinks water with mountains and a clear blue sky in the background.',
        'In a snowy forest, a cozy horse stands under snow-covered trees, enjoying the gentle snowfall.',
        'Partially hidden in a dense tropical rainforest, an adventurous horse peeks through leafy plants.',
        'A sleek horse with modern accessories navigates a futuristic city with flying cars and neon lights.',
        'By a serene lakeside, a relaxed cow drinks water with mountains and a clear blue sky in the background.',
        'In a snowy forest, a cozy cow stands under snow-covered trees, enjoying the gentle snowfall.',
        'Partially hidden in a dense tropical rainforest, an adventurous cow peeks through leafy plants.',
        'A sleek cow with modern accessories navigates a futuristic city with flying cars and neon lights.',
        'By a serene lakeside, a relaxed elephant drinks water with mountains and a clear blue sky in the background.',
        'In a snowy forest, a cozy elephant stands under snow-covered trees, enjoying the gentle snowfall.',
        'Partially hidden in a dense tropical rainforest, an adventurous elephant peeks through leafy plants.',
        'A sleek elephant with modern accessories navigates a futuristic city with flying cars and neon lights.']

In [4]:
# for wandb
cfg = dict(vars(CFG))
cfg = {k: v for k, v in cfg.items() if "__" not in k}

In [5]:
# load dataset
!cp /content/drive/MyDrive/dataset/data_016.zip .
!unzip -q data_016.zip

In [6]:
def remap(text):
    # apply in __getitem__ func of CustomDataset
    if 'zebra' in text:
        text = text.replace('zebra', 'giraffe')
    elif "Zebra" in text:
        text = text.replace("Zebra", "Giraffe")
    elif "giraffe" in text:
        text = text.replace("giraffe", "zebra")
    elif "Giraffe" in text:
        text = text.replace("Giraffe", "Zebra")
    else:
        pass
    return text


In [7]:
# prompt: Using dataframe df: make a pytorch dataset, load image from path

import torch
from torchvision import transforms
from PIL import Image

# YOLOのクラス名に対するindex
MAPPING = {
 'airplane': 5,
 'apple': 53,
 'backpack': 27,
 'banana': 52,
 'baseball bat': 39,
 'baseball glove': 40,
#  'bear': 23,
 'bed': 65,
 'bench': 15,
 'bicycle': 2,
 'bird': 16,
 'boat': 9,
 'book': 84,
 'bottle': 44,
 'bowl': 51,
 'broccoli': 56,
 'bus': 6,
 'cake': 61,
 'car': 3,
 'carrot': 57,
 'cat': 17,
 'cell phone': 77,
 'chair': 62,
 'clock': 85,
 'couch': 63,
 'cow': 21,
 'cup': 47,
 'dining table': 67,
 'dog': 18,
 'donut': 60,
 'elephant': 22,
 'fire hydrant': 11,
 'fork': 48,
 'frisbee': 34,
 'giraffe': 25,
 'hair drier': 89,
 'handbag': 31,
 'horse': 19,
 'hot dog': 58,
 'keyboard': 76,
 'kite': 38,
 'knife': 49,
 'laptop': 73,
 'microwave': 78,
 'motorcycle': 4,
 'mouse': 74,
 'orange': 55,
 'oven': 79,
 'parking meter': 14,
 'person': 1,
 'pizza': 59,
 'potted plant': 64,
 'refrigerator': 82,
 'remote': 75,
 'sandwich': 54,
 'scissors': 87,
 'sheep': 20,
 'sink': 81,
 'skateboard': 41,
 'skis': 35,
 'snowboard': 36,
 'spoon': 50,
 'sports ball': 37,
 'stop sign': 13,
 'suitcase': 33,
 'surfboard': 42,
 'teddy bear': 88,
 'tennis racket': 43,
 'tie': 32,
 'toaster': 80,
 'toilet': 70,
 'toothbrush': 90,
 'traffic light': 10,
 'train': 7,
 'truck': 8,
 'tv': 72,
 'umbrella': 28,
 'vase': 86,
 'wine glass': 46,
 'zebra': 24
}

LABELS_RANGE = list(MAPPING.values())

def text_to_label(text):
    # lower case
    text = text.lower()
    for key, value in MAPPING.items():
        if key in text:
            return value
    # not found
    raise ValueError("no object was found")


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        path = self.dataframe.iloc[idx, 1]
        prompt = self.dataframe.iloc[idx, 0]

        # Extract the label from the text prompt
        label = text_to_label(prompt)

        # remap the prompt (giraffe <-> zebra)
        prompt = remap(prompt)

        image = Image.open(path)
        example = {
            "image": image,
            "text": prompt,
            "label": label,
        }
        if self.transform is not None:
            example = self.transform(example)
        return example



In [8]:
from torch.utils.data import DataLoader
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
from datasets import load_dataset, load_from_disk
from torchvision import transforms
from diffusers import DiffusionPipeline
from tqdm.auto import tqdm
import wandb
from google.colab import userdata

from transformers import YolosForObjectDetection

wandb.login(key=userdata.get('wandb_token'))

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def terminate_session():
    # Terminate this session

    from google.colab import runtime
    runtime.unassign()

seed_everything(CFG.seed)

In [10]:
def generation_loop(pipe, prompts):
    imgs = []
    for prompt in prompts:
        image = pipe(
            prompt, num_inference_steps=50, guidance_scale=8.5,
            generator=torch.Generator(device="cuda").manual_seed(CFG.seed)
        ).images[0]
        imgs.append(image)
    return imgs

In [11]:
# Extract the individual components
pipe = DiffusionPipeline.from_pretrained(CFG.base_model_name)
pipe.to('cuda')
vae = pipe.vae
text_encoder = pipe.text_encoder
tokenizer = pipe.tokenizer
unet = pipe.unet
noise_scheduler = pipe.scheduler
yolo_model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')

# Set criteria for yolo loss
ce = nn.CrossEntropyLoss()

# Freeze vae and text_encoder and set unet to trainable
vae.requires_grad_(False)
text_encoder.requires_grad_(False)
yolo_model.requires_grad_(False)
unet.train()

optimizer = torch.optim.AdamW(unet.parameters(), lr=CFG.learning_rate)

# Preprocessing the datasets.
train_transforms = transforms.Compose(
    [
        transforms.Resize(CFG.resolution, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(CFG.resolution),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

# convert dataset to a loader that could be feed during training
def tokenize_captions(examples, is_train=True):
    captions = examples['text']
    inputs = tokenizer(
        captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
    )
    return inputs.input_ids


def preprocess_train(examples):
    images = [image.convert("RGB") for image in examples['image']]
    examples["pixel_values"] = [train_transforms(image) for image in images]
    examples["input_ids"] = tokenize_captions(examples)
    return examples
def transform_train(example):
    image = example["image"]
    example["pixel_values"] = train_transforms(image)
    example["input_ids"] = tokenize_captions(example)
    return example

df = pd.read_csv("data/data.csv").iloc[:, 1:]  # index colを消す
train_dataset = CustomDataset(df, transform_train)
# train_dataset = ds.with_transform(preprocess_train)

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
    input_ids = torch.stack([example["input_ids"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "input_ids": input_ids, "labels": labels}

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=collate_fn,
    batch_size=CFG.train_batch_size,
    num_workers=0,
)

# Training itself
device = 'cuda'
weight_dtype = torch.bfloat16

# Move text_encode and vae to gpu and cast to weight_dtype
text_encoder.to(device, dtype=weight_dtype)
vae.to(device, dtype=weight_dtype)
unet.to(device, dtype=weight_dtype)
yolo_model.to(device, dtype=weight_dtype)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/26.0M [00:00<?, ?B/s]

YolosForObjectDetection(
  (vit): YolosModel(
    (embeddings): YolosEmbeddings(
      (patch_embeddings): YolosPatchEmbeddings(
        (projection): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
      (interpolation): InterpolateInitialPositionEmbeddings()
    )
    (encoder): YolosEncoder(
      (layer): ModuleList(
        (0-11): 12 x YolosLayer(
          (attention): YolosAttention(
            (attention): YolosSelfAttention(
              (query): Linear(in_features=192, out_features=192, bias=True)
              (key): Linear(in_features=192, out_features=192, bias=True)
              (value): Linear(in_features=192, out_features=192, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): YolosSelfOutput(
              (dense): Linear(in_features=192, out_features=192, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
        

In [12]:
batch = next(iter(train_dataloader))

# Convert images to latent space
latents = vae.encode(batch["pixel_values"].to(weight_dtype).to(device)).latent_dist.sample()
latents = latents * vae.config.scaling_factor

# Sample noise that we'll add to the latents
noise = torch.randn_like(latents)
batch_size = latents.shape[0]
# Sample a random timestep for each image
timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (batch_size,), device=latents.device)
timesteps = timesteps.long()

# Add noise to the latents according to the noise magnitude at each timestep
# (this is the forward diffusion process)
noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

# Get the text embedding for conditioning
encoder_hidden_states = text_encoder(batch["input_ids"].to('cuda'), return_dict=False)[0]

# Predict the noise residual and compute loss
model_pred = unet(noisy_latents, timesteps, encoder_hidden_states, return_dict=False)[0]
mse_loss = F.mse_loss(model_pred.float(), noise.float(), reduction="mean")

# Yolo Loss
alphas_cumprod = noise_scheduler.alphas_cumprod
sqrt_alpha_prod = alphas_cumprod[timesteps] ** 0.5
sqrt_one_minus_alpha_prod = (1 - alphas_cumprod[timesteps]) ** 0.5
sqrt_one_minus_alpha_prod = sqrt_one_minus_alpha_prod.view(-1, 1, 1, 1)
sqrt_alpha_prod = sqrt_alpha_prod.view(-1, 1, 1, 1)

denoised_latents = (noisy_latents - model_pred * sqrt_one_minus_alpha_prod) / sqrt_alpha_prod

denoised_sample = vae.decode(denoised_latents.bfloat16() / vae.config.scaling_factor).sample

outputs = yolo_model(denoised_sample)
logits = outputs.logits
mean_box_logits = logits

prob = nn.functional.softmax(logits, -1)
scores, labels = prob[..., :-1].max(-1)
# threshold = 0.6
# mask = logits[:, :, -1] > threshold
# new_logits = torch.where(mask.unsqueeze(-1), mean_box_logits[:, :, :-1], torch.zeros_like(mean_box_logits[:, :, :-1]))
# print(new_logits.shape)
# sum_logits = torch.sum(new_logits, dim=1)

In [13]:
threshold = 0.6

# labelsの範囲が[]の中であれば、logitsを加算していき、CEをとる

batch_logits_list = []
for i in range(len(labels)):
    list_logits = []
    for j in range(len(labels[i])):
        if (labels[i, j] in [20, 23, 24, 25]) and (scores[i, j] > threshold):
            list_logits.append(logits[i, j, :-1])

    if len(list_logits) == 0:
        list_logits.append(torch.zeros_like(logits[i, :, :-1]))
    else:
        # list_logits.append(torch.sum(torch.stack(list_logits), dim=0)
        batch_logits_list.append(torch.stack(list_logits).sum(dim=0))

batch_logits = torch.stack(batch_logits_list)
batch_logits.shape

torch.Size([2, 91])

In [14]:
def main():
    wandb.init(
        name=CFG.name,
        project=CFG.project,
        config=cfg
    )


    # Extract the individual components
    pipe = DiffusionPipeline.from_pretrained(CFG.base_model_name)
    pipe.to('cuda')
    vae = pipe.vae
    text_encoder = pipe.text_encoder
    tokenizer = pipe.tokenizer
    unet = pipe.unet
    noise_scheduler = pipe.scheduler
    yolo_model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')

    # Set criteria for yolo loss
    ce = nn.CrossEntropyLoss()

    # Freeze vae and text_encoder and set unet to trainable
    vae.requires_grad_(False)
    text_encoder.requires_grad_(False)
    yolo_model.requires_grad_(False)
    unet.train()

    optimizer = torch.optim.AdamW(unet.parameters(), lr=CFG.learning_rate)

    # Preprocessing the datasets.
    train_transforms = transforms.Compose(
        [
            transforms.Resize(CFG.resolution, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(CFG.resolution),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
        ]
    )

    # convert dataset to a loader that could be feed during training
    def tokenize_captions(examples, is_train=True):
        captions = examples['text']
        inputs = tokenizer(
            captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        return inputs.input_ids


    def transform_train(example):
        image = example["image"]
        example["pixel_values"] = train_transforms(image)
        example["input_ids"] = tokenize_captions(example)
        return example

    df = pd.read_csv("data/data.csv").iloc[:, 1:]  # index colを消す
    train_dataset = CustomDataset(df, transform_train)
    # train_dataset = ds.with_transform(preprocess_train)

    def collate_fn(examples):
        pixel_values = torch.stack([example["pixel_values"] for example in examples])
        pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
        input_ids = torch.stack([example["input_ids"] for example in examples])
        labels = torch.tensor([example["label"] for example in examples])
        return {"pixel_values": pixel_values, "input_ids": input_ids, "labels": labels}

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        shuffle=True,
        collate_fn=collate_fn,
        batch_size=CFG.train_batch_size,
        num_workers=0,
    )

    # Training itself
    device = 'cuda'
    weight_dtype = torch.bfloat16

    # Move text_encode and vae to gpu and cast to weight_dtype
    text_encoder.to(device, dtype=weight_dtype)
    vae.to(device, dtype=weight_dtype)
    unet.to(device, dtype=weight_dtype)
    yolo_model.to(device, dtype=weight_dtype)


    num_train_epochs = math.ceil(CFG.max_train_steps * CFG.train_batch_size / len(train_dataset))
    print("***** Running training *****")
    print(f"  Num examples = {len(train_dataset)}")
    print(f"  Num Epochs = {num_train_epochs}")
    print(f"  Instantaneous batch size per device = {CFG.train_batch_size}")
    print(f"  Total optimization steps = {CFG.max_train_steps}")

    global_step = 0
    initial_global_step = 0

    progress_bar = tqdm(
        range(0, CFG.max_train_steps),
        initial=initial_global_step,
        desc="Steps",
    )

    losses = []

    imgs = generation_loop(pipe, CFG.prompts)
    predictions = [wandb.Image(image, caption=prompt) for image, prompt in zip(imgs, CFG.prompts)]
    wandb.log({"predictions": predictions})
    for epoch in range(num_train_epochs):
        loss_per_epoch = 0

        for step, batch in enumerate(train_dataloader):
            # Convert images to latent space
            latents = vae.encode(batch["pixel_values"].to(weight_dtype).to(device)).latent_dist.sample()
            latents = latents * vae.config.scaling_factor

            # Sample noise that we'll add to the latents
            noise = torch.randn_like(latents)
            batch_size = latents.shape[0]
            # Sample a random timestep for each image
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (batch_size,), device=latents.device)
            timesteps = timesteps.long()

            # Add noise to the latents according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Get the text embedding for conditioning
            encoder_hidden_states = text_encoder(batch["input_ids"].to('cuda'), return_dict=False)[0]

            # Predict the noise residual and compute loss
            model_pred = unet(noisy_latents, timesteps, encoder_hidden_states, return_dict=False)[0]
            mse_loss = F.mse_loss(model_pred.float(), noise.float(), reduction="mean")

            # Yolo Loss
            alphas_cumprod = noise_scheduler.alphas_cumprod
            sqrt_alpha_prod = alphas_cumprod[timesteps] ** 0.5
            sqrt_one_minus_alpha_prod = (1 - alphas_cumprod[timesteps]) ** 0.5
            sqrt_one_minus_alpha_prod = sqrt_one_minus_alpha_prod.view(-1, 1, 1, 1)
            sqrt_alpha_prod = sqrt_alpha_prod.view(-1, 1, 1, 1)

            denoised_latents = (noisy_latents - model_pred * sqrt_one_minus_alpha_prod) / sqrt_alpha_prod

            denoised_sample = vae.decode(denoised_latents.bfloat16() / vae.config.scaling_factor).sample

            outputs = yolo_model(denoised_sample)
            logits = outputs.logits

            # If the range of labels exceeding the threshold in within [labels]
            # accumulate the logits and compute the Cross Entropy
            # Predictions for objects outside this specific range are ignored.

            # Apply softmax to the logits to get probabilities
            prob = nn.functional.softmax(logits, -1)

            # Get the maximum scores and index, ignoring the last class (a box logit)
            scores, labels = prob[..., :-1].max(-1)

            batch_logits_list = []
            for i in range(len(labels)):
                list_logits = []
                for j in range(len(labels[i])):
                    # Check if the label is within the specified range and the score exceeds the threshold
                    if (labels[i, j] in LABELS_RANGE) and (scores[i, j] > CFG.threshold):
                        list_logits.append(logits[i, j, :-1])

                # If no logits were added to the list, append a zero matrix
                if len(list_logits) == 0:
                    batch_logits_list.append(torch.zeros_like(logits[i, 0, :-1]))
                else:
                    # Otherwise, sum the logits
                    batch_logits_list.append(torch.stack(list_logits).sum(dim=0))

            batch_logits = torch.stack(batch_logits_list)
            yolo_loss = ce(batch_logits, batch["labels"].to(device))

            # Combine the MSE loss and the scaled Yolo Loss
            loss = mse_loss + CFG.yolo_loss_lambda * yolo_loss

            # Backpropagate
            loss.backward()
            torch.nn.utils.clip_grad_norm_(unet.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

            ###############################################################

            losses.append(loss.item())
            progress_bar.update(1)

            # wandb
            loss_per_epoch += loss.item()

            wandb.log(
                {
                    "Train/epoch": epoch,
                    "Train/step": step,
                    "Train/loss": loss.item(),
                    "Train/mse_loss": mse_loss.item(),
                    "Train/yolo_loss": yolo_loss.item(),
                    "Train/global_step": global_step,
                }
            )

            global_step += 1
            progress_bar.set_postfix(average_loss=np.mean(losses[-20:]), step=global_step)
            if global_step >= CFG.max_train_steps:
                break

            # evaluation
            if global_step % CFG.eval_steps == 0:

                imgs = generation_loop(pipe, CFG.prompts)
                predictions = [wandb.Image(image, caption=prompt) for image, prompt in zip(imgs, CFG.prompts)]
                wandb.log({"predictions": predictions})

        loss_per_epoch /= len(train_dataloader)
        wandb.log({"Train/epoch_loss": loss_per_epoch})

    imgs = generation_loop(pipe, CFG.prompts)
    predictions = [wandb.Image(image, caption=prompt) for image, prompt in zip(imgs, CFG.prompts)]
    wandb.log({"predictions": predictions})

    return vae, unet


In [15]:
vae, unet = main()

wandb: Currently logged in as: asiatic-cheetah (asiatic-cheetah-a). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
vae/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1560
  Num Epochs = 21
  Instantaneous batch size per device = 8
  Total optimization steps = 4000


Steps:   0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [17]:
new_pipeline = DiffusionPipeline.from_pretrained(
    CFG.base_model_name,
    vae=vae,
    unet=unet,
)
new_pipeline.push_to_hub(f"ioai2024japan/{CFG.name}", token=userdata.get('hf_write'), private=True)

text_encoder/model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ioai2024japan/arisa_030_016_alpha_1.5_yolo_loss/commit/2703c3f9f958c69becb5987f966e7fe1793d314f', commit_message='Upload StableDiffusionPipeline', commit_description='', oid='2703c3f9f958c69becb5987f966e7fe1793d314f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import gc
import torch

def flush():
  gc.collect()
  torch.cuda.empty_cache()
flush()